## 基本的 LLMs 工作流程

本笔记本演示了三个简单的 LLMs 工作流程。他们会权衡成本或延迟以获得可能提高的任务性能：

1. **提示链接**：将任务分解为连续的子任务，其中每个步骤都建立在先前结果的基础上
2. **并行化**：在多个 LLM 之间分配独立的子任务以进行并发处理
3. **路由**：根据输入特征动态选择专门的LLM路径
注意：这些是示例实现，旨在演示核心概念，而不是生产代码。

In [1]:
from concurrent.futures import ThreadPoolExecutor

from util import extract_xml, llm_call

In [ ]:
def chain(input: str, prompts: list[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = llm_call(f"{prompt}\nInput: {result}")
        print(result)
    return result


def parallel(prompt: str, inputs: list[str], n_workers: int = 3) -> list[str]:
    """Process multiple inputs concurrently with the same prompt."""
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]


def route(input: str, routes: dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""
    # First determine appropriate route using LLM with chain-of-thought
    print(f"\nAvailable routes: {list(routes.keys())}")
    selector_prompt = f"""
    Analyze the input and select the most appropriate support team from these options: {list(routes.keys())}
    First explain your reasoning, then provide your selection in this XML format:

    <reasoning>
    Brief explanation of why this ticket should be routed to a specific team.
    Consider key terms, user intent, and urgency level.
    </reasoning>

    <selection>
    The chosen team name
    </selection>

    Input: {input}""".strip()

    route_response = llm_call(selector_prompt)
    reasoning = extract_xml(route_response, "reasoning")
    route_key = extract_xml(route_response, "selection").strip().lower()

    print("Routing Analysis:")
    print(reasoning)
    print(f"\nSelected route: {route_key}")

    # Process input with selected specialized prompt
    selected_prompt = routes[route_key]
    return llm_call(f"{selected_prompt}\nInput: {input}")

## 用法示例

以下是演示每个工作流程的实际示例：

1. 结构化数据提取和格式化的链式工作流程

2. 利益相关者影响分析的并行化工作流程

3. 客户支持票证处理的路由工作流程

In [3]:
# 示例 1：结构化数据提取和格式化的链式工作流程，每一步都会逐步将原始文本转换为格式化表格

data_processing_steps = [
    # 第一步：提取数值和指标
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",
    # 第二步：标准化数值格式，将数值转换为百分比或小数形式
    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",
    # 第三步：按数值降序排序
    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",
    # 第四步：格式化为 markdown 表格
    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |""",
]

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

print("\nInput text:")
print(report)
formatted_result = chain(report, data_processing_steps)


Input text:

Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.


Step 1:
Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
92: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
8%: customer churn (previous)
$43: new user acquisition cost
78%: product adoption rate
87: employee satisfaction
34%: operating margin

Step 2:
Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
92%: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
8%: customer churn (previous)
43.0: new user acquisition cost
78%: product adoption rat

In [4]:
# 示例 2：利益相关者影响分析的并行化工作流程，同时对多个利益相关者群体进行影响分析

stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",
    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",
    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",
    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions""",
]

impact_results = parallel(
    """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
    stakeholders,
)

for result in impact_results:
    print(result)
    print("+" * 80)

Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
Based on the input provided, here is an analysis of how market changes (inflationary pressures, rapid technological advancement, and the green transition) will impact your **Customers**, along with recommended strategic actions.

---

### **Stakeholder Analysis: Customers**

#### **1. Impact Area: Price Sensitivity**
*Market Context: Rising raw material costs and inflation are forcing price increases across most industries.*
*   **Specific Impacts:**
    *   **Reduced Brand Loyalty:** Customers are more likely to "shop around" or switch to generic/cheaper alternatives to save costs.
    *   **Longer Sales Cycles:** H

In [5]:
# 示例 3：客户支持票证处理的路由工作流程，根据内容分析将支持请求发送给适当的团队

support_routes = {
    # 1. 账单支持团队
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant

    Keep responses professional but friendly.

    Input: """,
    # 2. 技术支持团队
    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed

    Use clear, numbered steps and technical details.

    Input: """,
    # 3. 账户支持团队
    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time

    Maintain a serious, security-focused tone.

    Input: """,
    # 4. 产品支持团队
    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help

    Be educational and encouraging in tone.

    Input: """,
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John""",
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike""",
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = route(ticket, support_routes)
    print(response)
    print("+" * 80)

Processing support tickets...


Ticket 1:
----------------------------------------
Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Connecting to LLM: Model=google/gemini-3-flash-preview, Base_URL=https://openrouter.ai/api/v1
Routing Analysis:

The user is experiencing a login failure specifically related to an "invalid password" error and is requesting help to regain access to their profile. While there is a technical element to the error, issues involving credentials, authentication, and identity verification are primarily handled by the Account team. The urgency is high due to a deadline, but the core intent is account recovery.


